# Batch UCB
- use your favorite kernel to generate distance matrix
- clustering (k-metriod)
- large clustering numbers: reject clustering for which the max UCB is smaller other clustering's LCB; then choose sequences in the remaining clusterings by UCB.
- small clustering numbers, e.g. 90, choose one sequence from each clustering by UCB.

In [1]:
# direct to proper path
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from collections import defaultdict
import math
import json

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import PairwiseKernel, DotProduct, RBF 
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import KFold
from sklearn_extra.cluster import KMedoids

from codes.embedding import Embedding
from codes.environment import Rewards_env
from codes.ucb import GPUCB, Random
from codes.evaluations import evaluate, plot_eva
from codes.regression import *
from codes.kernels_for_GPK import Spectrum_Kernel, Sum_Spectrum_Kernel, WeightedDegree_Kernel
from codes.batch_ucb import *

from ipywidgets import IntProgress
from IPython.display import display
import warnings
%matplotlib inline

/home/admin-u6015325/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
Path = '../../data/firstRound_Microplate_normTrue_formatSeq_logTrue.csv'

known_df = pd.read_csv(Path)
known_df.head()

,Unnamed: 0,Name,Group,RBS,RBS6,Rep1,Rep2,Rep3,Rep4,Rep5,Rep6,AVERAGE,STD
0,0,RBS_1by1_0,reference,TTTAAGAAGGAGATATACAT,AGGAGA,1.616261,1.814182,1.760954,2.186207,2.028863,1.831982,1.881293,0.225819
1,1,RBS_1by1_1,bps_noncore,CTTAAGAAGGAGATATACAT,AGGAGA,1.166174,1.337018,1.417248,1.493800,1.713526,1.644568,1.425553,0.201725
2,2,RBS_1by1_2,bps_noncore,GTTAAGAAGGAGATATACAT,AGGAGA,0.604551,0.751384,0.851987,0.514929,0.577299,0.795227,0.660030,0.137994
3,3,RBS_1by1_3,bps_noncore,ATTAAGAAGGAGATATACAT,AGGAGA,1.221264,1.466278,1.270212,1.341040,1.395030,1.175433,1.338765,0.097386
4,4,RBS_1by1_4,bps_noncore,TCTAAGAAGGAGATATACAT,AGGAGA,1.160566,1.579025,1.171829,1.590670,1.411255,1.756862,1.382669,0.210012


In [3]:
top_n = Top_n_ucb(known_df, kernel_name='WD_Kernel_Shift', 
                normalise_kernel_flag='True', embedding='label', alpha=0.1,
                eva_metric=mean_squared_error, l_list=[6], s=1, use_samples_for_train = False,
                rec_size=90, beta=1)

In [4]:
top_n_rec_df = top_n.run_experiment()

/home/admin-u6015325/git/SynbioML/codes/regression.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_df['label'] = self.train_df['AVERAGE']


X train shape:  (176, 20)
X test shape:  (3961, 20)
create kernel instance
finish creating kernel instance
After centering
(176, 176)
[[ 8.4116465   9.25504389  9.25504389 ...  0.98690334  1.18858545
   2.69696202]
 [ 9.25504389 12.09844129 11.09844129 ...  1.83030074  2.03198285
   3.54035942]
 [ 9.25504389 11.09844129 12.09844129 ...  1.83030074  2.03198285
   3.54035942]
 ...
 [ 0.98690334  1.83030074  1.83030074 ...  8.4669221   1.08527087
   2.14126649]
 [ 1.18858545  2.03198285  2.03198285 ...  1.08527087  8.46552441
   0.9977105 ]
 [ 2.69696202  3.54035942  3.54035942 ...  2.14126649  0.9977105
   8.45846802]]
(array([-1.40580651e+00,  1.39821179e-01,  1.73832604e-01,  2.16138561e-01,
        3.31992318e-01,  3.92248830e-01,  4.00732752e-01,  4.11414366e-01,
        4.32184373e-01,  4.42749969e-01,  4.59975905e-01,  4.71970114e-01,
        4.76190476e-01,  5.07406683e-01,  5.65095184e-01,  5.94869205e-01,
        6.42054553e-01,  6.71060948e-01,  6.94480642e-01,  7.00407066e-01,

LinAlgError: ("The kernel, WD_Shift_Kernel(sigma_0=1e-10), is not returning a positive definite matrix. Try gradually increasing the 'alpha' parameter of your GaussianProcessRegressor estimator.", '76-th leading minor of the array is not positive definite')

In [ ]:
gpbucb = GP_BUCB(known_df, kernel_name='WD_Kernel_Shift', 
                normalise_kernel_flag='True', embedding='label', alpha=0.1,
                eva_metric=mean_squared_error, l_list=[6], s=1,use_samples_for_train = False,
                rec_size=90, beta=1)

In [ ]:
gpbucb_rec_df = gpbucb.run_experiment()